# 遗传算法

## 0. 参数

In [210]:
import numpy as np
import random
import os

maxElement = 28
mutationRate = .03  # 变异的概率
crossoverRate = .8   # 交叉互换的概率
inverseRate = .1  # 发生逆行变换的概率
inverseLength = 32  # 逆行变换的序列长度
invertRate = .1  # 发生倒影变换的概率
invertLength = 32  # 发生倒影变换的长度
transpositionRate = .1  # 移调变换的概率
transpositionLevel = 2  # 移调变换的音程

iterations = 30
generation = []  # 当前一代中的所有个体
genSize = 10
sampleSize = 5

crossCnt = 0  # 记录总共的交叉互换次数
mutationCnt = 0  # 记录总共的变异次数

# 上一个组用的fitness function里的参数
alpha = .1
beta = .1
ai = []
bi = []

## 1. 定义individual类

In [182]:

class Individual:
    '''
    一个种群中的每一个个体
    属性:
        chromosome: list of int, 染色体（音符）序列
        fitness: fitness value, 按照给定的fitness function计算出的fitness值
    方法: 
        getFitness(): 得到fitness值
        getChromosome(): 得到chromosome序列
        setChromosome(newChrom): 修改chromosome序列
    '''
    
    
    def __init__(self, chromosome):
        '''
        chromosome: list of int
        '''
        self.chromosome = chromosome
        self.fitness = self.calFitness()
        if self.fitness > Individual.bestFitness:
            Individual.bestChromosom = self.chromosome
            Individual.bestFitness = self.fitness
            
    
    def calFitness(self):
        '''
        计算fitness的函数, 待修改
        '''
        Fibonacci = {0, 1, 2, 3, 5, 8, 13}
        # melody score
        melody = 0
        for i in range(len(self.chromosome) - 1):
            if abs(self.chromosome[i] - self.chromosome[i+1]) in Fibonacci:
                melody += 1
        # harmonics score
        harmonics = 0
        for i in range(0, len(self.chromosome), 8):
            if self.chromosome[i] == 0 or self.chromosome[i] == maxElement:
                harmonics -= 10
            if i + 8 > len(self.chromosome):
                break
            u = np.mean(self.chromosome[i:i+8])
            v = np.var(self.chromosome[i:i+8])
            harmonics -= alpha * (abs(u - ai[int(i/8)])) + beta * (abs(v - bi[int(i/8)]))
        fitness = melody + harmonics
        return fitness
    
    
    # 得到fitness属性
    def getFitness(self):
        return self.fitness
    
    # 得到chromosome属性
    def getChromosome(self):
        return self.chromosome
    
    # 修改chromosome属性
    def setChromosome(self, newChrom):
        self.chromosome = newChrom


## 2. 定义进化中的行为

In [183]:
def crossover(individuals):
    '''
    交叉互换函数
    以crossoverRate的概率进行交叉互换
    Parameters:
    ----------
        individuals: list of 2 individuals, [individual_1, individual_2]
     
    Returns:
    --------
        list of 2 chromosome, 交叉互换后的两个染色体序列
        
    '''
    global crossoverRate, crossCnt
    if random.random() > crossoverRate:
        return [individuals[0].getChromosome(), individuals[1].getChromosome()]
    crossCnt += 1
    length = len(individuals[0].getChromosome())
    splitpoint = random.randint(1, length-2)
    return [individuals[0].getChromosome()[:splitpoint]+individuals[1].getChromosome()[splitpoint:],
            individuals[1].getChromosome()[:splitpoint]+individuals[0].getChromosome()[splitpoint:]]
    

def mutation(gen):
    '''
    变异函数
    以mutationRate的概率进行交叉互换
    Parameters:
    ----------
        gen: generation, list of individuals, 某一代种群的全体
     
    Returns:
    --------
        gen: 变异后的generation
    '''
    global maxElement
    global mutationRate, mutationCnt
    for ind in gen:
        newChrom = ind.getChromosome()
        for i in range(len(newChrom)):
            if random.random() < mutationRate:
                mutationCnt += 1
                newChrom[i] = random.randint(0, maxElement)
        ind.setChromosome(newChrom)
    return gen


def transform(chromomsome):
    '''
    进行逆行、倒影、移调变换
    Parameters:
    ----------
        chromomsome: list of int 
        
    Returns:
    ----------
        chromomsome: chromomsome after transformation, list of int 
    '''
    
    def eliminate_28(chromomsome):
        '''
        replace "28" with the last note
        '''
        for i in range(len(chromomsome)):
            if chromomsome[i] == 28:
                chromomsome[i] = chromomsome[i-1]
        return chromomsome


    def restore(chromomsome):
        '''
        restore the representation form with 28 indicating the note is the same as the last
        '''
        for i in range(len(chromomsome)-1, 0, -1):
            if chromomsome[i] == chromomsome[i-1]:
                chromomsome[i] = 28
        return chromomsome
    
    # 先把延长符变成对应的音级符号，便于进行下列计算
    chromomsome = eliminate_28(chromomsome)
    # 逆行变换
    global inverseRate, inverseLength
    if random.random() < inverseRate:
        start = random.randint(0,len(chromomsome)-inverseLength)
        chromomsome[start:start+inverseLength] = reversed(chromomsome[start:start+inverseLength])
    # 倒影变换
    global invertRate, invertLength
    if random.random() < invertRate:
        start = random.randint(0,len(chromomsome)-invertLength)
        for i in range(start, start+invertLength):
            chromomsome[i] = maxElement - chromomsome[i]
    # 移调变换
    global transpositionRate, transpositionLevel
    if random.random() < transpositionRate:
        for i in range(len(chromomsome)):
            chromomsome[i] = (chromomsome[i] + transpositionLevel) % maxElement
    # 再把连续的音符变回延长号
    chromomsome = restore(chromomsome)
    
    return chromomsome


def creatNextGen():
    '''
    生成下一代
    '''
    global maxElement
    global generation, genSize, sampleSize
    newGen = []
    # 交叉互换
    # 找到fitness最高的parents，每次产生两个新的individual，所以要int(genSize/2)次
    for i in range(0, int(genSize/2)):
        # 在原来的generation中选取一些样本
        subgen = random.sample(generation, sampleSize)
        bestFitness = [-1000, -1000]
        bestIndividuals = [None, None]
        # 找到随机样本中fitness最高的两个parents
        for ind in subgen:
            if ind.getFitness() > bestFitness[0] or ind.getFitness() > bestFitness[1]:
                if bestFitness[0] > bestFitness[1]:
                    bestFitness[1] = ind.getFitness()
                    bestIndividuals[1] = ind
                else:
                    bestFitness[0] = ind.getFitness()
                    bestIndividuals[0] = ind
        # 对这两个parents交叉互换
        newChromosomes = crossover(bestIndividuals)
        newGen.append(Individual(newChromosomes[0]))
        newGen.append(Individual(newChromosomes[1]))
    # 变异
    newGen = mutation(newGen)
    # 逆行、倒影、移调变换
    for ind in newGen:
        ind.setChromosome(transform(ind.chromosome))
    
    generation = newGen  # 新的generation


## 3. 进化

In [212]:
def genetic_algorithm(duration):
    '''
    初始化generation, 并按照设置好的iterations次数迭代
    '''
    global generation
    global crossCnt, mutationCnt
    global alpha, beta, ai, bi
    generation = []
    Individual.bestChromosom = None
    Individual.bestFitness = -1000
    crossCnt = 0
    mutationCnt = 0
    
    alpha = 0.02
    beta = 0.02
    ai = []
    bi = []
    for i in range(0, duration, 8):
        if i + 8 > duration:
            break
        ai.append(random.randint(3, 12))
        bi.append(random.uniform(5, 30))

    # 从外部获得初始generation
    directory = '../music/'
    fileNames = os.listdir(directory)
    for i in fileNames:
        generation.append(Individual([j for j in np.loadtxt(directory + i)]))

    # 进行进化算法
    
    for i in range(iterations):
        print(Individual.bestFitness)
        print(Individual.bestChromosom)
        creatNextGen()
        
    

In [213]:
genetic_algorithm(32)

22.27584716777548
[17.0, 15.0, 20.0, 28.0, 28.0, 28.0, 18.0, 17.0, 15.0, 28.0, 28.0, 28.0, 20.0, 18.0, 17.0, 15.0, 20.0, 28.0, 28.0, 17.0, 15.0, 28.0, 28.0, 28.0, 10.0, 28.0, 17.0, 28.0, 15.0, 28.0, 28.0, 28.0]
22.37740966777548
[17.0, 15.0, 20.0, 28, 28, 28, 18.0, 17.0, 15.0, 28, 28, 28, 20.0, 18.0, 17.0, 13, 20.0, 28, 28, 17.0, 15.0, 28, 28, 28, 10.0, 28, 17.0, 28, 28, 28, 28, 28]
22.37740966777548
[17.0, 15.0, 20.0, 28, 28, 28, 18.0, 17.0, 15.0, 28, 28, 28, 20.0, 18.0, 17.0, 13, 20.0, 28, 28, 17.0, 15.0, 28, 28, 28, 10.0, 28, 17.0, 28, 28, 28, 28, 28]
22.409597167775484
[19.0, 17.0, 22.0, 28, 28, 28, 20.0, 19.0, 17.0, 28, 28, 28, 22.0, 20.0, 19.0, 15, 22.0, 28, 28, 19.0, 17.0, 28, 24, 28, 20.0, 23, 13.0, 28, 28, 3, 28, 28]
22.409597167775484
[19.0, 17.0, 22.0, 28, 28, 28, 20.0, 19.0, 17.0, 28, 28, 28, 22.0, 20.0, 19.0, 15, 22.0, 28, 28, 19.0, 17.0, 28, 24, 28, 20.0, 23, 13.0, 28, 28, 3, 28, 28]
22.409597167775484
[19.0, 17.0, 22.0, 28, 28, 28, 20.0, 19.0, 17.0, 28, 28, 28, 22.0, 20.